# Axisim Punch-Through Shear Test

This example couples two domains via an zero-thickness interface. 

State - 2021-07-19 [RC]:

- Slide34 included with not verified material parameters. The push through test is running for a small step size
- Add the force control boundary conditions in analogy to single facet example
- Change of the parameters to brittle response leads to convergence problems at the transition to inelastic state - check the range which converges.
- Either initial or secant stiffness has been used     

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt

In [2]:
from ibvpy.api import TStepBC
from ibvpy.bcond import BCSlice
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

# Material models

In [5]:
from bmcs_matmod.slide.vslide_34 import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic

# FE discretization

In [14]:
n_y_e = 10
n_inner_x_e = 5
n_outer_x_e = 5
L_x = 200.0
R_in = 100.0
R_out = 200.0
xd_inner = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_outer = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=28000, nu=0.3)
m2 = MATS3DElastic(E=28000, nu=0.3)

xd12 = XDomainFEInterface(
    I=xd_inner.mesh.I[1:-1, -1],
    J=xd_outer.mesh.I[1:-1, 0],
    fets=FETS1D52ULRH()
)

In [86]:
#bond_m = Slide34(E_T=10000, E_N=10000)
material_params = dict(
    E_T=10000, gamma_T=4000, K_T=0, S_T=0.000001, c_T=2, bartau=5, 
    E_N=20000, S_N=0.00005, c_N = 1, m = 0.008, f_t=5, f_c=150, f_c0=100, eta=0.0 # 0.5
)
bond_m = Slide34(**material_params)

In [87]:
axisym_x = BCSlice(slice=xd_inner.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_outer_y = BCSlice(slice=xd_outer.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_inner_y = BCSlice(slice=xd_inner.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=-0.005)
# free_inner_y = BCSlice(slice=xd_inner.mesh[:, 0, :, 0],
#                   var='u', dims=[1], value=0)
bc1 = [axisym_x, fixed_outer_y, loaded_inner_y]

In [88]:
m = TStepBC(
    domains=[(xd_inner, m1),
             (xd_outer, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
    #        'damage': Vis3DStateField(var='omega_a'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.05
s.tstep.fe_domain.serialized_subdomains

In [89]:
#bond_m.interact()

In [90]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 0.05(1), 
	 0.10(15), 
	 0.15(16), 
	 0.20(16), 
	 0.25(16), 
	 0.30(16), 
	 0.35(16), 
	 0.40(16), 
	 0.45(16), 
	 0.50(16), 
	 0.55(16), 
	 0.60(16), 
	 0.65(16), 
	 0.70(16), 
	 0.75(16), 
	 0.80(16), 
	 0.85(16), 
	 0.90(16), 
	 0.95(16), 
	 1.00(16), 



# Plot the punch-through curve

In [92]:
F_to = m.hist.F_t
U_to = m.hist.U_t

In [93]:
F_to.shape

(21, 264)

In [94]:
F_outer_t = np.sum(F_to[:, fixed_outer_y.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_inner_y.dofs], axis=-1)
U_inner_top_t = np.average(U_to[:, loaded_inner_y.dofs], axis=-1)
#U_inner_bot_t = np.average(U_to[:, free_inner_y.dofs], axis=-1)

In [95]:
_, ax = plt.subplots(1,1)
ax.plot(-U_inner_top_t, F_outer_t, label='F_outer(U_top)')
#ax.plot(-U_inner_top_t, F_inner_t, label='F_inner(U_top)')
#ax.plot(-U_inner_bot_t, F_inner_t, label='F_inner(U_bot)')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [96]:
if False:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'strain'
    strain_viz = Viz3DTensorField(vis3d=m.hist['strain'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

In [97]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}

In [98]:
sig_tx = EpsSig_t['sig_pi'].reshape(21,-1)
tau_tx = EpsSig_t['tau_pi_x'].reshape(21,-1)
omega_tx = EpsSig_t['omega_T'].reshape(21,-1)

In [99]:
x_m = xd12.x_Eia[:,:,0].flatten()

In [100]:
_, ax = plt.subplots(1,1)
idx = -1
ax_omega = ax.twinx()
ax.plot(x_m, sig_tx[idx,:], color='blue', label='sig')
ax.plot(x_m, tau_tx[idx,:], color='red', label='tau')
ax_omega.plot(x_m, omega_tx[idx,:], color='red', label='tau')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …